In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from gensim.models import phrases, word2vec, KeyedVectors
import nltk
import numpy as np
import codecs
from collections import Counter
from nltk.corpus import stopwords
import argparse
import os
import string
import re
from scipy.stats import ttest_ind

In [14]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
root_dir = "/content/drive/My Drive/Polygence/Audrey/"

## Load data

In [16]:
# load data
import pandas as pd
data = pd.read_csv(root_dir + "/data/all_articles_dependencies.csv")
data.head()

,Unnamed: 0,source,author,title,url,publishedAt,content,coref_resolved,Bennet_verbs,Bennet_adjectives,Biden_verbs,Biden_adjectives,Blasio_verbs,Blasio_adjectives,Booker_verbs,Booker_adjectives,Bullock_verbs,Bullock_adjectives,Buttigieg_verbs,Buttigieg_adjectives,Castro_verbs,Castro_adjectives,Delaney_verbs,Delaney_adjectives,Gabbard_verbs,Gabbard_adjectives,Gillibrand_verbs,Gillibrand_adjectives,Harris_verbs,Harris_adjectives,Inslee_verbs,Inslee_adjectives,Klobuchar_verbs,Klobuchar_adjectives,Moulton_verbs,Moulton_adjectives,O'Rourke_verbs,O'Rourke_adjectives,Ryan_verbs,Ryan_adjectives,Sanders_verbs,Sanders_adjectives,Steyer_verbs,Steyer_adjectives,Warren_verbs,Warren_adjectives,Williamson_verbs,Williamson_adjectives,Yang_verbs,Yang_adjectives,Bloomberg_verbs,Bloomberg_adjectives,month
0,0,abc-news,John Verhovek,5 takeaways from the 2nd night of the Democrat...,https://abcnews.go.com/Politics/takeaways-2nd-...,2019-08-01T03:31:25Z,\nEven before all the candidates took the stag...,\nEven before all the candidates took the stag...,had 1\nsaid 1,NaN,said 3\nsparred 1\ncriticized 1\nhit 1\nlearne...,NaN,warned 1\nbelieve 1,NaN,said 2\ncriticized 1\nimplored 1\nwas 1,NaN,NaN,NaN,NaN,NaN,said 2\nwent 1\nargued 1,NaN,NaN,NaN,said 1,NaN,challenged 2\ntook 1\nsaid 1\nmade 1,concerned 1,were 1\ncriticized 1\nsaid 1\noffered 1\navoid 1,NaN,hit 1\nimplored 1\nwas 1\nsaid 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,was 1\nsaid 1,NaN,NaN,NaN,aug
1,1,abc-news,ABC NEWS,Fact-checking Democratic candidates on the iss...,https://abcnews.go.com/Politics/democratic-deb...,2019-08-01T01:29:38Z,\nHere's ABC News' fact check of the second of...,\nHere's ABC News' fact check of the second of...,NaN,NaN,said 4\nwas 2\nwrote 2\nresponded 2\nvoted 2\n...,NaN,NaN,NaN,inherited 1\nput 1\nam 1\nhire 1\nwrote 1\nstr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'m 1,| 1,is 3\nthink 1\nbelieve 1\nraised 1,NaN,said 3\nkept 2\nacknowledging 2\nran 2\ndefend...,| 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,said 1\nbelieve 1,NaN,NaN,NaN,aug
2,2,abc-news,Kendall Karson and Elizabeth Thomas,What to expect from 2nd night of Democratic de...,https://abcnews.go.com/Politics/democratic-deb...,2019-07-31T20:01:36Z,\nAfter Sens. Bernie Sanders and Elizabeth War...,\nAfter Sens. Bernie Sanders and Elizabeth War...,said 5\nchimed 1\nresponded 1\ncontinued 1\ncr...,NaN,said 11\nresponded 4\nwas 3\nis 2\nasked 2\npu...,old 1,said 3\noppose 1\nmake 1\npressed 1\nasked 1\n...,NaN,said 5\nthink 2\nendorsed 2\ncriticized 1\nwan...,glad 1,NaN,NaN,NaN,NaN,said 6\nbelieve 2\nwant 2\npushed 1\nthink 1\n...,NaN,NaN,NaN,said 4\nasks 1\nbelieves 1\nbacked 1\nshared 1...,old 1,said 6\nasked 2\nfind 1\nwant 1\nend 1\nput 1\...,NaN,said 7\nput 2\npushed 2\nam 2\nresponded 2\nin...,NaN,said 4\nhit 1\npushing 1\ntakes 1\napproach 1\...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fended 1\nwrote 1\nhave 1,introduced 1,NaN,NaN,impose 1,NaN,has 2\ntells 2\nsaid 1,NaN,said 5\njoined 1\nthink 1\nasked 1\nresponded ...,NaN,NaN,NaN,jul
3,3,abc-news,Soo Rin Kim,2020 candidates spend millions on online ads a...,https://abcnews.go.com/Politics/love-hate-rela...,2019-07-27T10:22:00Z,\nDemocrats vying for presidential election in...,\nDemocrats vying for presidential election in...,burned 1,NaN,become 1\nincluding 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spent 1,NaN,NaN,NaN,NaN,NaN,filed 1\nsuspending 1,NaN,NaN,NaN,spent 1,NaN,spent 1,NaN,said 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spent 1\nsaid 1,NaN,NaN,NaN,said 2\nreleased 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jul
4,4,abc-news,Rick Klein and MaryAlice Parks,The Note: Blue wave continues ride through Ken...,https://abcnews.go.com/Politics/note-blue-wave...,2019-11-06T11:05:10Z,\nThe TAKE with Rick Klein\n \nIt might just b...,\nThe TAKE with Rick Klein\n \nIt might just b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,asked 2\ndraw 1\nboost 1\npoised 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [0]:
# combine all text
a = data["coref_resolved"]
all_data = " ".join(data["coref_resolved"])

## Clean text

In [0]:
punct_chars = list((set(string.punctuation) | {'»', '–', '—', '-',"­", '\xad', '-', '◾', '®', '©','✓','▲', '◄','▼','►', '~', '|', '“', '”', '…', "'", "`", '_', '•', '*', '■'} - {"'"}))
punct_chars.sort()
punctuation = ''.join(punct_chars)
replace = re.compile('[%s]' % re.escape(punctuation))
sno = nltk.stem.SnowballStemmer('english')
printable = set(string.printable)
stopwords = set(stopwords.words('english'))

In [0]:
def clean_text(text, stem=False):
    # lower case
    text = text.lower()
    # eliminate urls
    text = re.sub(r'http\S*|\S*\.com\S*|\S*www\S*', ' ', text)
    # substitute all other punctuation with whitespace
    text = replace.sub(' ', text)
    # replace all whitespace with a single space
    text = re.sub(r'\s+', ' ', text)
    # strip off spaces on either end
    text = text.strip()
    # make sure all chars are printable
    text = ''.join([c for c in text if c in printable])
    words = text.split()
    # remove numeric
    words = [w for w in words if not w.isdigit()]
    # stem non stopwords
    if stem:
      words = [sno.stem(w) if w not in stopwords else w for w in words]
    return words

In [27]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# split text into sentences (nltk.sent_tokenize)
sentences = nltk.sent_tokenize(all_data)

In [0]:
# clean each sentence using the function clean_text()
cleaned_sentences = []
for sent in sentences:
  cleaned_sentences.append(clean_text(sent))

## Create model

In [0]:
# Create vocabulary model
bigrams = phrases.Phrases(all_sentences, min_count=5, delimiter=b' ', common_terms=stopwords)

In [0]:
# Create vocabulary of bigrams
print("Creating vocabulary...")
data = bigrams[all_sentences]
vocab = [w for sent in data for w in sent]
vocab = [w for w, count in Counter(vocab).most_common() if count >= 5]

# Save vocab
with codecs.open(os.path.join(root_dir + "models", 'word2vec_vocab.txt'), 'w', encoding='utf-8') as f:
    f.write('\n'.join(vocab))

In [0]:
# Run word2vec model with bootstrapping
sample_size = int(0.8 * len(all_sentences))
NUM_RUNS = 20
for run_idx in range(NUM_RUNS):
    print("Run #%d" % run_idx)
    data = bigrams[np.random.choice(all_sentences, sample_size, replace=True)]
    model = word2vec.Word2Vec(data, size=200, window=5, sg=1, min_count=5, workers=10)
    model.wv.save(os.path.join(root_dir + "models", "word2vec_model_" + str(run_idx) + '.wv'))

In [0]:
# Run for each source
def run_word2vec_for_source(data, source):
  data = data[data["source"] == source]
  all_text = " ".join(data['coref_resolved'])
  sents = nltk.sent_tokenize(all_text)
  print(len(sents))
  all_sentences = [clean_text(sent) for sent in sents]
  bigrams = phrases.Phrases(all_sentences, min_count=5, delimiter=b' ', common_terms=stopwords)
  # Create vocabulary of bigrams
  print("Creating vocabulary...")
  data = bigrams[all_sentences]
  vocab = [w for sent in data for w in sent]
  vocab = [w for w, count in Counter(vocab).most_common() if count >= 5]

  # Save vocab
  with codecs.open(os.path.join(root_dir + "models", source + '_vocab.txt'), 'w', encoding='utf-8') as f:
      f.write('\n'.join(vocab))

  # Run word2vec model for source
  model = word2vec.Word2Vec(data, size=200, window=5, sg=1, min_count=5, workers=10)
  model.wv.save(os.path.join(root_dir + "models", source + "_model.wv"))

In [0]:
sources = data.groupby("source").filter(lambda x: len(x) > 200)["source"].unique()
sources

## Load models

In [0]:
def get_models(filelist):
    model_files = [f for f in filelist if f.endswith('.wv')]
    models = [KeyedVectors.load(fname, mmap='r') for fname in model_files]
    return models

In [33]:
filelist = []
for subdir, dirs, files in os.walk(root_dir + "models/word2vec_bootstrap"):
  for file in files:
    filelist.append(os.path.join(subdir, file))
models = get_models(filelist)
len(models)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


12

In [0]:
# Get vocab (intersection of all vocab)
vocab = set(models[0].vocab)
for m in models:
    vocab &= set(m.vocab)
len(vocab)
idx2word = {i: w for i, w in enumerate(list(vocab))}

# Look at word similarity

Check out this documentation: https://radimrehurek.com/gensim/models/word2vec.html

### Find closest words to a query / set of queries

Hint: use the `model.similarity()` function

In [0]:
def get_closest(queries, models, vocab, idx2word):
    cosines = []
    for m in models:
        cosines.append([np.mean([m.similarity(q, word) for q in queries]) for word in vocab])
    cosines = np.mean(np.array(cosines), axis=0)
    return [(idx2word[idx], cosines[idx]) for idx in cosines.argsort()[-20:][::-1]]

In [0]:
def filter_words(words, vocab):
    words = set(words)
    not_in_vocab = words - vocab
    if not_in_vocab:
        print("Not in vocab:")
        print(not_in_vocab)
    return list(words - not_in_vocab)

In [38]:
queries = ["power"] # this should be a list of queries (it can also be just one query)

 # Remove queries not in vocab
queries = filter_words(queries, vocab)

print("Getting most similar words...")
closest = get_closest(queries, models, vocab, idx2word)
for (w, c) in closest:
    print("%s %.2f" % (w, c))

Getting most similar words...


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


power 1.00
personal gain 0.50
powers 0.49
abusing 0.48
authority 0.48
office to solicit 0.48
corrupting 0.47
trump abused 0.46
wield 0.45
levers 0.45
public trust 0.45
leverage 0.45
pressuring a foreign 0.45
exert 0.45
accused of abusing 0.45
personal interests 0.44
wields 0.44
speak truth 0.44
subvert 0.44
powers of the presidency 0.43


#### Compare the similarity of a query to two words / two sets of words (e.g. candidate names)

For example, compare the similarity of the word "power" to two different candidate names (lowercase).

In [0]:
def get_cosines(name1, name2, words1, words2, queries, models):
    df_w1 = []
    df_w2 = []
    df_q = []
    df_type = []
    df_pvals = []
    for key, values in queries.items():
        for q in values:
            vals1 = [m.similarity(word1, q) for m in models for word1 in words1]
            vals2 = [m.similarity(word2, q) for m in models for word2 in words2]
            df_w1.append(np.mean(vals1))
            df_w2.append(np.mean(vals2))
            df_q.append(q)
            df_type.append(key)
            df_pvals.append(ttest_ind(vals1, vals2)[1])
    df = pd.DataFrame({name1: df_w1, name2: df_w2, 'query': df_q, 'word category': df_type, "p value": df_pvals})
    return df

In [0]:
queries = {"achievement": ["achieved", "success", "successful"]}

#{"power": ["powerful", "power", "authority", "powers", "influence"]}
words1 = ["biden", "joe", "joe biden"]
words2 =  ["sanders", "bernie", "bernie sanders"]
# Remove queries not in vocab
words1 = filter_words(words1, vocab)
words2 = filter_words(words2, vocab)
for k, v in queries.items():
    queries[k] = filter_words(v, vocab)

In [43]:
get_cosines("biden", "sanders", words1, words2, queries, models)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,biden,sanders,query,word category,p value
0,0.168594,0.172994,success,achievement,0.716380
1,0.178649,0.200800,successful,achievement,0.048377
2,0.220130,0.202273,achieved,achievement,0.220743
